# K2 Platform - Getting Started Guide

**Focus**: Simple, copy-pastable code examples showing platform usage

---

## What You'll Learn

- Check platform health and connectivity
- Monitor live data streaming (Binance)
- Query data via REST API
- Test production resilience features

---

## Sections

1. **Platform Status** - Check system health
2. **Live Data Pipeline** - Binance streaming status
3. **API Demonstration** - REST API queries
4. **Production Features** - Resilience demo

---

## Prerequisites

- Docker services running: `docker compose up -d`
- API accessible at: `http://localhost:8000`

In [ ]:
# Import required libraries
import subprocess
import re

import pandas as pd
import requests

# Platform configuration
API_BASE = "http://localhost:8000"
GRAFANA_URL = "http://localhost:3000"

print("K2 Demo - Libraries imported successfully")
print(f"Using pandas {pd.__version__}")

In [ ]:
# Check platform status
print("K2 Platform Status Check\n")

# API Health Check
try:
    response = requests.get(f"{API_BASE}/health", timeout=5)
    if response.status_code == 200:
        health = response.json()
        print(f"API Status: {health.get('status')}")
        print(f"Version: {health.get('version')}")

        # Check dependencies
        for dep in health.get('dependencies', []):
            status = "healthy" if dep.get('status') == 'healthy' else "unhealthy"
            latency = dep.get('latency_ms', 0)
            print(f"  {dep['name']}: {status} ({latency:.1f}ms)")
    else:
        print(f"API Status: HTTP {response.status_code}")
except Exception as e:
    print(f"API Error: {e}")

# Check available symbols
try:
    symbols_response = requests.get(f"{API_BASE}/v1/symbols", timeout=5)
    if symbols_response.status_code == 200:
        symbols = symbols_response.json()
        print(f"\nAvailable Symbols: {len(symbols)}")
        for symbol in symbols[:5]:
            print(f"  - {symbol}")
        if len(symbols) > 5:
            print(f"  ... and {len(symbols) - 5} more")
except Exception as e:
    print(f"Symbols Error: {e}")

In [ ]:
# Check Binance streaming status
print("Binance Streaming Status\n")

try:
    # Get streaming logs
    result = subprocess.run(
        ["docker", "logs", "k2-binance-stream", "--tail", "50"],
        capture_output=True, text=True, timeout=10
    )

    if result.returncode == 0:
        # Strip ANSI color codes from logs
        ansi_escape = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')
        clean_logs = ansi_escape.sub('', result.stdout)

        # Parse streaming metrics from clean logs
        trades_streamed = 0
        last_symbol = "Unknown"
        last_price = 0

        # Find all trade counts and get the highest (most recent)
        trade_matches = re.findall(r'trades_streamed=(\d+)', clean_logs)
        if trade_matches:
            trades_streamed = max([int(m) for m in trade_matches])

        # Find last symbol from the most recent line
        symbol_matches = re.findall(r'symbol=(\w+)', clean_logs)
        if symbol_matches:
            last_symbol = symbol_matches[-1]

        # Find last price from the most recent line
        price_matches = re.findall(r'last_price=([\d.]+)', clean_logs)
        if price_matches:
            last_price = float(price_matches[-1])

        print(f"Total Trades: {trades_streamed:,}")
        print(f"Last Symbol: {last_symbol}")
        print(f"Last Price: ${last_price:,.2f}")

        if trades_streamed > 0:
            print("\nStreaming is ACTIVE and processing data")
        else:
            print("\nNo recent streaming activity")
    else:
        print("Could not fetch streaming logs")

except Exception as e:
    print(f"Streaming check error: {e}")

print("\nAccess Points:")
print(f"  Grafana: {GRAFANA_URL}")
print(f"  API Docs: {API_BASE}/docs")

In [ ]:
# Demonstrate API capabilities
print("API Demonstration\n")

try:
    # Test trades endpoint
    trades_response = requests.get(
        f"{API_BASE}/v1/trades?limit=5",
        headers={'X-API-Key': 'k2-dev-api-key-2026'},
        timeout=5
    )
    if trades_response.status_code == 200:
        response_data = trades_response.json()
        trades = response_data.get('data', [])
        print(f"Recent Trades ({len(trades)} available):")

        for trade in trades[:3]:
            if isinstance(trade, dict):
                symbol = trade.get('symbol', 'Unknown')
                price = trade.get('price', 0)
                quantity = trade.get('quantity', 0)
                timestamp = trade.get('timestamp', 'Unknown')
                print(f"  - {symbol}: ${price:.2f} x {quantity} @ {timestamp}")
    else:
        print(f"Trades endpoint returned status {trades_response.status_code}")

    # Test metrics endpoint
    metrics_response = requests.get(f"{API_BASE}/metrics", timeout=5)
    if metrics_response.status_code == 200:
        metrics_text = metrics_response.text
        k2_metrics = [line for line in metrics_text.split('\n') if 'k2_' in line]
        print(f"\nPlatform Metrics: {len(k2_metrics)} K2-specific metrics collected")

        # Show sample metrics
        for metric in k2_metrics[:3]:
            metric_name = metric.split('{')[0] if '{' in metric else metric.split(' ')[0]
            print(f"  - {metric_name}")

except Exception as e:
    print(f"API demo error: {e}")

print("\nAPI Access:")
print(f"  Documentation: {API_BASE}/docs")
print(f"  Health check: {API_BASE}/health")
print(f"  Metrics: {API_BASE}/metrics")

In [ ]:
# Demonstrate resilience features
print("Production Resilience Demo\n")

try:
    # Run degradation simulation
    print("Running resilience simulation...")

    result = subprocess.run(
        ["python", "../scripts/demo_degradation.py", "--quick"],
        capture_output=True, text=True, timeout=20
    )

    if result.returncode == 0:
        output = result.stdout

        # Extract key information
        print("Resilience features working:")

        lines = output.split('\n')
        for line in lines:
            if 'Degradation Level' in line and '│' in line:
                print(f"  {line.strip()}")
            elif 'Behavior at' in line:
                print(f"  {line.strip()}")
            elif 'Demo Complete' in line:
                print(f"  {line.strip()}")
                break

    else:
        print("Resilience demo encountered issues")

except subprocess.TimeoutExpired:
    print("Resilience demo timed out (normal for quick mode)")
except Exception as e:
    print(f"Resilience demo error: {e}")

print("\nProduction Features:")
features = [
    "- 5-level degradation cascade (NORMAL → SOFT → GRACEFUL → AGGRESSIVE → CIRCUIT_BREAK)",
    "- Priority-based load shedding (always process critical data)",
    "- Auto-recovery with hysteresis (prevents flapping)",
    "- Dead Letter Queue (failed message handling)",
    "- Circuit breaker patterns (prevent cascade failures)",
    "- 83 Prometheus metrics (comprehensive monitoring)",
]

for feature in features:
    print(f"  {feature}")